## Introduction

This is an investigation of various Approximate Nearest Neighbors (ANN) algorithms for use in machine learning.  The main core of each test is as follows:  

* A query is sampled from the set of compounds that target EGFR and are deemed to be relatively active (the query is 20% of this set)
* An ANN model is created and trained on the remaining EGFR compounds (both active and inactive) and a database of 200000 other compounds
* The query is run on the model, and the quality of the test is determined by the proportion of neighbors recieved that are members of the original active EGFR compounds

The variations between tests come from the type of model and the hyperparameters for each model.  Each combination is also repeated for five trials, using a different random seed for the query sampling.  The different models are from the Python package scikit-learn, Facebook's Faiss algorithms, and Spotify's Annoy algorithms.  There is also an analysis utilizing scikit-learn's clustering algorithms with Faiss to offer an additional perspective.

## Sections
 1. [sklearn](sklearn.ipynb)
 2. [faiss](Faiss.ipynb)
 3. [annoy](Annoy.ipynb)
 4. [comparing all 3](all3.ipynb)
 5. [clustering](Clustering.ipynb)

## Conclusions

### sklearn

* All algorithms return a similar quality. Query times for all algorithms except ball_tree are reasonable (around 5 seconds), but not super fast.  Ball_tree is longer (around 30 seconds).  Within the faster algorithms, kd_tree is fastest, followed by brute-force with euclidean distance metric, followed by brute-force with cosine distance metric.
* The tree algorithms take longer (around 15 seconds) than the brute-force (around 0.1 seconds) to build, which makes sense as trees are being created, as opposed to the very little setup required for the brute-force model.  Note that build time is largely inconsequential, as an application of this would most likely include build a model once, and then running many queries on the same model.
* From the information gathered, it appears as if the kd_tree algorithm is the best of the four, although not by very much.

### faiss

* All algorithms return a similar quality.  Query times vary from 0.01 seconds for HNSWFlat to around 0.7 seconds for FlatL2 and FlatIP. The times are overall faster than those from sklearn while maintaining a similar level of quality.
* The build times are about 1 second with the exception of HNSWFlat, which takes about 44 seconds.  This extra time seems worth the extremely fast query time available from this algorithm.  
* From the information gathered, it appears as if the HNSWFlat algorithm is the best of the five, although not by very much.

### annoy

* All algorithms return a similar quality. The hamming distance metric seems to have a slightly lower quality, as well as more variability in query time and build time. 
* The build times for the annoy algorithms are significantly greater than those of other algorithms.  This is because Annoy does not support model-building with an entire dataframe.  Rather, each vector must be added individually to the model.  Again, this is not likely to be impactful, but is worth considering.
* From the information gathered, it appears as if the hamming distance metric is the worth of the four.  The other 3 are all good.

### all 3

* We can split the algorithms into three different classes based on build time.  Call class 1 those with a mean build time less than 5 seconds, class 2 those with a mean build time between 5 and 100 seconds, and class 3 those with a mean build time greater than 100 seconds.
    * Class 1:
        * sklearn/brute/cosine
        * sklearn/brute/euclidean
        * faiss/FlatL2
        * faiss/FlatIP
        * faiss/LSH
        * faiss/PQ
    * Class 2:
        * sklearn/ball_tree/euclidean
        * sklearn/kd_tree/euclidean
        * faiss/HNSWFlat
    * Class 3:
        * annoy/angular
        * annoy/euclidean
        * annoy/hamming
        * annoy/manhattan
        
* We can also do the same for query time.  Call class 1 those with a mean query time less than 1 second, class 2 those with a mean query time between 1 and 10 seconds, and class 3 those with a mean query time greater than 10 seconds.
    * Class 1:
        * faiss/FlatL2
        * faiss/FlatIP
        * faiss/HNSWFlat
        * faiss/LSH
        * faiss/PQ
    * Class 2:
        * sklearn/brute/cosine
        * sklearn/brute/euclidean
        * sklearn/kd_tree/euclidean
        * annoy/angular
        * annoy/euclidean
        * annoy/hamming
        * annoy/manhattan
    * Class 3:
        * sklearn/ball_tree_euclidean

|            | Build Time | Class 1 (t<5)                              | Class 2 (5<t<100)           | Class 3 (t>100)                                             |
|------------|------------|:------------------------------------------:|:---------------------------:|:----------------------------------------------------------:|
| Query Time |            |                                              |                             |                                                             |
| Class 1 (t<1)|            | faiss/FlatL2 <br> faiss/FlatIP <br> faiss/LSH <br> faiss/PQ | faiss/HNSWFlat              |                                                             |
| Class 2 (1<t<10)|            | sklearn/brute/cosine <br> sklearn/brute/euclidean | sklearn/kd_tree/euclidean   | annoy/angular <br> annoy/euclidean <br> annoy/hamming <br> annoy/manhattan |
| Class 3 (t>10)|            |                                              | sklearn/ball_tree/euclidean |                                                             |

Lower classes are better, so we can create a rough hierarchy: faiss (except HNSWFlat), then skearn brute and faiss HNSWFlat, then sklearn kd_tree, etc.